<h1>Training Mario Using Different Reinforcement Learning Methods

Antonio Casas z5312681, Raymond Chan z5061486, Peter Boylan z5309087, Austin Hoe z5414020, Sixuan Yang z5442076

## 1. Introduction & Objectives
Our project aims to maximise World 1.1 completion rate in Super Mario Bros by training a Reinforcement Learning (RL) Mario agent. Mario’s task is learning how to make the optimal moves to reach the flag by navigating through or evading various obstacles such as enemies, holes, and pipes. 

Unfortunately, reinforcement learning algorithms can be computationally demanding and their efficiency is significantly influenced by the choice of hyperparameters, the reinforcement learning algorithm’s complexity, and data pre-processing. Therefore, finding the best combination of these factors can lead to more time and computation-sparing learning and improved performance of the model. From the numerous RL methods available, we chose to focus on two methods, double-deep Q learning (DDQL) and proximal policy optimisation (PPO), which are value [11] and policy-based [12] RL algorithms respectively.

## 2.  Setup Libraries for RL Tasks

##### <b>EMULATOR ENVIRONMENT<b>:
- ``gym`` is a [toolkit](https://www.gymlibrary.dev/index.html) by OpenAI designed for testing different reinforcement learning algorithms in different gaming environments (e.g. ATARI or NES). 
- ``gym.spaces.Box`` provides a continue observation space for our Mario.
- ``gym.wrappers.FrameStack`` aids in providing a temporal element to training, where multiple observations are stacks so that historical information of Mario's position can contribute to learning. Find documentation on wrappers [here](https://www.gymlibrary.dev/api/wrappers/).
- ``gym_super_mario_bros`` supplies the Super Mario Bros environment (from which we can specify worlds, levels & visualisation settings)
- ``nes_py.wrappers.JoypadSpace`` from [nes-py](https://pypi.org/project/nes-py/) allows for the mapping of joystick and button actions to a discrete action space. In this investigation, we simplify the action space to just right and up+right movement.

##### <b>NEURAL NETWORKS<b>:
- ``torch.nn`` and ``torch.nn.functional`` are methods and collections of functions that are used to create deep neural networks (such as the CNN architecture we create in the DDQL section) and perform tensor manipulation respectively.
- ``tensordict.TensorDict`` allows us to handle dictionaries within PyTorch tensors.
- ``TensorDictReplayBuffer`` and ``LazyMemmapStorage`` from ``torchrl.data``: used to cache and load tensor replays from within training epochs.

#### 2.1 - Import Dependencies on Virtual Environment

In [52]:
%%bash
pip install gym-super-mario-bros==7.4.0
pip install tensordict==0.2.0
pip install torchrl==0.2.0


pip install gym==0.21.0
pip install gym-super-mario-bros==7.4.0
pip install nes-py==8.2.1
pip install pyglet==1.5.21
pip install stable-baselines3==1.5.0
pip install setuptools==65.5.0 pip==21
pip install stable-baselines3==1.5.0
pip install torch torchvision
# pip install torch==1.11.0 torchvision==0.12.0
pip install 'shimmy>=0.2.1'
pip install tensorboard==2.7.0 tensorboardX==2.4
pip install tensordict
pip install torchrl

  Using cached torch-2.0.1-cp311-none-macosx_10_9_x86_64.whl (143.1 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.1.1
    Uninstalling torch-2.1.1:
      Successfully uninstalled torch-2.1.1


#### 2.2 - Import Necessary Libraries

In [55]:
'''OPEN-AI GYM AND NES EMULATOR MODULES/METHODS'''
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace

'''IMPORT PYTORCH TO CREATE A NEURAL NETWORK MODEL'''
import torch
from torch import nn
from torchvision import transforms as T
import torch.nn.functional as F

'''MODULES TO BE USED FOR VISUALISATION, DATA PROCESSING AND METRIC EVALUATION'''
import numpy as np
from pathlib import Path
import datetime, os
from tensordict import TensorDict
# from torchrl.data import TensorDictReplayBuffer, LazyMemmapStorage
import time, datetime
import matplotlib.pyplot as plt

'''IMPORT STABLEBASELINES FOR PPO ALGORITHM'''
# from stable_baselines3 import PPO
# from stable_baselines3.common.callbacks import EvalCallback


'''HIDE WARNINGS'''
import warnings
warnings.filterwarnings('ignore')

## 3.  Exploratory Analysis of RL Tasks

### Environment Pre-Processing

We implemented our models within an OpenAI Gym Environment using a virtual NES emulator, then using a library called super-mario-bros-gym to manipulate the original NES environment. This library allows us to simplify our action space, so the agent can only choose from going forward and jumping. Subsequently, we simplified the environment (as seen in Figure 1) to boost the computational efficiency of the strenuous training process, removing redundancies in background and object details. 

Because each state is represented by a ``[3, 240, 256]`` dimensional matrix, we need to downsample and downscale the environment so that the model can be trained as swiftly as possible. For instance, we don't care about the details of the Mario agent, thus, we can reduce him to just a dynamic rectangle moving in an undetailed grayscale world. We used Wrapper classes with the help of ``gym.wrappers`` to achieve the following:

- Use ``GrayScaled`` eliminates unnecessary color channels, such that we reduce dimensionality from ``[3, 240, 256]`` to ``[1, 240, 256]``.

- Use ``ResizeFrames`` to downsample each observation state into a square image: ``[1, 84, 84]``

- Use ``FrameSkipper`` to skip n-intermediate frames as not every frame contains unique, valuable, information. In order to not lose reward information, we aggregate the rewards accumulated in the otherwise skipped frame into the returned n-th frame.

- Use ``FrameStack`` to condense multiple frames into a single observation, creating a temporal element to training. We can now recall Mario's actions in the previous several frames. Therefore, the final environment shape is ``[n, 84, 84]`` where n = the size of each frame stack.

All of these classes and methods are used within a convenient ``create_environment`` function that takes the world, level, skip, stack and action space as input params). 


<img src = visuals/env_comparison.png width = 550px>
<b>Figure 1:</b> (1) version-1 removes the background whereas (2) version-3 simplifies the game into a series of blocks. [15]


#### <b>We faced the following challenges when constructing our RL models:<b>
    
- Integrating multiple dependencies such as gym and stable-baselines-3 libraries due to compatibility issues.
    - Thus, we decided to implement the Mario agent and DDQN classes manually.
- Training time: our final model took 22 hours to train for 1.5 million steps, which was sufficient enough to consistently win World 1.1. We saved and loaded the model onto multiple devices to attempt to further improve performance. 
    - As such, our computational power was insufficient to train Mario to win on any further levels (we attempted 1.2, 1.3 and 2.1). 
- We faced some compatibility issues with gym wrappers and custom-made wrappers which were difficult to amend. 

In [63]:
class GrayScaled(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        observation_shape = self.observation_space.shape[:2]
        '''NORMALISED PIXEL INTENSITIES TO [0,1] RATHER THAN [0,255] TO SIMPLIFY INPUT TO CNN'''
        self.observation_space = Box(low=0, high=1, shape=observation_shape, dtype=np.uint8)

    def change_dims(self, observation):
        '''
        DIMENSIONS ARE CURRENTLY [HEIGHT, WIDTH, COLOR CHANNELS], BUT WE NEED TO
        RESHAPE INTO A [C,H,W] ARRAY
        '''
        observation = np.transpose(observation, (2, 0, 1))
        observation = torch.tensor(observation.copy(), dtype=torch.float)
        return observation

    def observation(self, observation):
        observation = self.change_dims(observation)
        '''USE TORCHVISION TRANSFORMATION METHOD'''
        transform = T.Grayscale()
        observation = transform(observation)
        return observation
    
class FrameSkipper(gym.Wrapper):
    def __init__(self, env, skip):
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        '''
        TO NOT LOSE REWARDS, THEY ARE ACCUMULATED EVEN IF THE FRAME IS SKIPPED
        THIS MEANS THAT STEPS ARE STILL TAKEN, BUT THE SPATIAL COMPLEXITY IS 
        REDUCED BY ONLY RETURNING THE N-TH FRAME.
        '''
        reward_total = 0.0
        for i in range(self._skip):
            observation, reward, done, info = self.env.step(action)
            reward_total += reward
            if done:
                break
        return observation, reward_total, done, info


class ResizeFrames(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        self.shape = (shape, shape) #84x84 IN THIS CASE
        observation_shape = self.shape + self.observation_space.shape[2:]
        '''NORMALISED PIXEL INTENSITIES TO [0,1] RATHER THAN [0,255] TO SIMPLIFY INPUT TO CNN'''
        self.observation_space = Box(low=0, high=1, shape=observation_shape, dtype=np.uint8)
    def observation(self, observation):
        '''ANTI-ALIASING REMOVES JAGGED/STEPPED EDGES OFTEN CREATED VIA RESIZING TRANSFORMATIONS'''
        transforms = T.Compose(
            [T.Resize(self.shape, max_size=None, antialias=True)]
        )
        '''REMOVE THE BATCH DIMENSION'''
        observation = transforms(observation).squeeze(0)
        return observation

def create_environment(version = '3', world = '1', stage = '1', action_space = [["right"], ["right", "A"]], skip = 4, num_stack = 4):
    if gym.__version__ < '0.26':
        env = gym_super_mario_bros.make(f"SuperMarioBros-{world}-{stage}-v{version}")
    else:
        '''LATER VERSIONS REQUIRE US TO SPECIFY COMPATIBILITY AND RENDER MODE'''
        env = gym_super_mario_bros.make(f"SuperMarioBros-{world}-{stage}-v{version}", render_mode='rgb', apply_api_compatibility=True)
    
    '''AFTER INITIALISING THE ENVIRONMENT, WE REDUCE ACTION SPACE COMPLEXITY AND APPLY ALL IMAGE PRE-PROCESSING TECHNIQUES'''
    env = JoypadSpace(env, action_space)
    env = FrameSkipper(env, skip = skip)
    env = GrayScaled(env)
    env = ResizeFrames(env, shape=84)
    
    '''FRAMESTACK IS A WRAPPER ALREADY PROVIDED BY THE GYM LIBRARY'''
    env = FrameStack(env, num_stack=num_stack)
    return env

## 4. Models & Methods

In the literature, Klein [14] shows that after training for 5000 iterations (that is, until Mario died or reached the end of the level), an agent was able to win about 90% of the time. We used his discount factor of 0.6, learning rate of 0.8 and epsilon-greedy upper bound of 0.3 as inspiration for our hyperparameter tuning. In this project, however, we focus on maximising the rewards obtained by the model while minimising the training time as each team member only had access to a CPU.<br>

Optimising training time enhances its versatility and transferability, making it potentially applicable to other similar tasks or environments. From their superior results within the literature as well as their widespread use in the context of ATARI and NES games emulated for RL tasks, we employed Double Deep Q-Learning (DDQL) and Proximal Policy Optimization (PPO) <br>

[The DDQN algorithm](https://arxiv.org/pdf/1509.06461) introduces a mechanism that employs two separate networks: the online network and the target network. The online network learns and updates the Q-values, while the target network, a delayed copy of the online network, provides stable Q-value targets ($Q_{online}$ vs $Q_{target}$). This is to promote symmetrical learning (contrary to the greedy overestimation bias present in regular DQN). This decoupling of action selection and action evaluation helps reduce over-optimistic value estimates, leading to more accurate Q-values and better policies. For DDQL, we implemented a CNN architecture and agent class that explores and exploits the action space, and stores and recalls previous states. DDQL is particularly appropriate for training Mario because of its non-stationary nature, where the delayed updating of the target network reduces variation and overestimation in Q-value predictions, thereby resulting in higher stability and likelihood for faster convergence. <br>

\begin{align}{TD}_e = Q_{online}^*(s,a)\end{align} <br>
\begin{align}a' = argmax_{a} Q_{online}(s', a)\end{align} <br>
\begin{align}{TD}_t = r + \gamma Q_{target}^*(s',a')\end{align} <br>

Where $Q^*$ is the optimal value under an optimal policy, TD<sub>e</sub> is the temporal difference estimation and TD<sub>t</sub> is the temporal different target value. DDQL is a form of TD learning.

As we sample from Mario's replay buffer, the online network is updated as follows:

\begin{align}\theta_{online} \leftarrow \theta_{online} + \alpha \nabla(TD_e - TD_t)\end{align}

Contrastingly, the target network doesn't undergo backpropagation, so it is updated as follows:
\begin{align}\theta_{target} \leftarrow \theta_{online}\end{align}

Where θ<sub>online</sub> and θ<sub>target</sub> are the neural network parameters for the online and offline networks respectively. 


On the other hand, [PPO](https://arxiv.org/pdf/1707.06347.pdf) is an optimization method that seeks to improve the policy while ensuring that the new policy doesn’t deviate drastically from the old one. What makes PPO different to DDQN and other value function methods is the use of an 'advantage function' that attempts to gauge how much 'better' or 'worse' an action is relative to the expected value. PPO maintains stability by adding a constraint to the ‘surrogate objective function’ (SOF), which penalises changes in the policy that are too large. This approach is crucial for achieving optimal results, especially if the old policy was near-optimal. As such, the negative consequences of stochasticity and complexity in the Mario environment can be mitigated through the stability provided by the SOF. PPO excels in processing high-dimensional inputs, which is a feature highly pertinent to managing the multi-dimensional pixel and temporal tensors required for training our Mario agent. In the context of Mario RL Learning, a 'bad' action means detrimental consequences (i.e., Mario's death), thus we want to minimise bias while simultaneously reducing variance between estimates.<br>
This is a high level description of what the stable-baselines-3 PPO method does:
1. The current policy is executed to collect 'trajectories', which is a set of action, state, reward tuples.
2. Advantages (A) are computed using the generalised advantage estimation function (GAE).
3. The value functions are updated using mean squared error calculations via backpropagation after a forward pass through a CNN.
4. Policy gradients ∇θ are computed on the objective function J(θ) using advantage estimates (A). 
5. The policy parameters are updated by selecting the action that increases the expected value (via argmax).

We manipulated the following hyperparameters:
- TAU: the trade-off hyperparameter responsible for modulating variance and bias in the advantage function. High TAU means low bias and high variance (and vice versa).
- BETA = the coefficient to the entropy function H(Pi), where a larger beta means higher entropy and thus a reduction in deterministic tendencies.
- CLIPPING EPSILON = controls the extent of clipping to prevent training destabilisation as a result of significant policy changes. This occurs in the 'surrogate objective function' where the new and old policies are compared and replaced.
- NUM GLOBAL STEPS = total number of steps across all parallel instances.
- NUM PARALLEL PROCESSES = total number of parallel instances of the agent occurring simulataneously.


#### 4.1 - Double Deep Q-Learning

In [64]:
'''
MANUAL IMPLEMENTATION OF DDQL ALGORITHM
'''

class DDQL(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        channels, height, width = input_dim
        
        '''INITIALISE THE ONLINE AND TARGET NETWORKS'''
        self.online = self.cnn_build(channels, output_dim)
        self.target = self.cnn_build(channels, output_dim)

        '''THE STATE OF THE TARGET NETWORK IS INITIALLY EQUAL TO THE ONLINE NETWORK PRIOR TO TRAINING'''
        self.target.load_state_dict(self.online.state_dict())

        '''THE TARGET NETWORK IS NEVER UPDATED VIA BACKPROPAGATION, BUT THROUGH INTERMITTENTLY TAKING THE ONLINE NETWORK'S PARAMS'''
        for p in self.target.parameters():
            p.requires_grad = False

    def forward(self, _input, model):
        if model == "online":
            return self.online(_input)
        else:
            return self.target(_input)

    def cnn_build(self, channels_dim, output_dim):
        '''
        BUILD A SIMPLE CNN WITH 2 CONVOLUTIONAL LAYERS, RELU ACTIVATION FUNCTIONS
        TO REDUCE INPUT MAGNITUDES, AND A MAX POOLING LAYER TO REDUCE DIMENSIONALITY.
        2 FULLY CONNECTED LAYERS ARE USED AT THE END TO CHOOSE ONE OF TWO ACTIONS (RIGHT OR UP/RIGHT)
        '''
        return nn.Sequential(
            nn.Conv2d(in_channels=channels_dim, out_channels=32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels= 32, out_channels= 64, kernel_size = 4, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(in_features = 3136, out_features = 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )

#### 4.2 - Proximal Policy Optimisation

In [65]:
# PPO

def ppo_learning(env, tensorboard_log, h_dict):
    '''
    INPUTS:
    ``env`` - the pre-processed Mario game environment
    ``tensorboard_log`` - the directory path to save a log to evaluation metrics
    ``h_dict`` - a dictionary with tunable hyperparameters
    '''
    learning_rate, gae_lambda, n_steps, batch_size = h_dict['learning_rate'], h_dict['tau'], h_dict['n_steps'], h_dict['batch_size']
    n_epochs, gamma, entropy_change, clip_range = h_dict['n_epochs'], h_dict['gamma_ppo'], h_dict['beta'], h_dict['clipping_epsilon']
    policy = h_dict['policy']
    
    '''BASECASE PPO ALGORITHM FROM THE LITERATURE'''
    # return PPO('CnnPolicy', env, verbose=1, tensorboard_log=tensorboard_log, learning_rate=0.000001, 
    #         n_steps=512) 

    return PPO(policy=policy, env=env,
            learning_rate=learning_rate, n_steps=n_steps,
            batch_size=batch_size, n_epochs=n_epochs,
            gamma=gamma, gae_lambda=gae_lambda,
            clip_range=clip_range,
            ent_coef=gamma,
            tensorboard_log=tensorboard_log, verbose=1)

#### 4.3 - Create An Agent Class (For DDQL)

We created a class called ``Mario`` who can act according to the learned policy, remember/cache/recall experiences (i.e. state, action, reward, subsequent state) used to update and learn the optimal policy.

There are two types of actions, exploitation (using a greedy policy such as Q-Learning) and exploration (using epsilon-greedy).


In [75]:
class Mario:
    def __init__(self, state_dim, n_possible_actions, h_dict, use_cuda = False):
        """
        Initialise the agent.

        Inputs:
        state_dim(``tuple``): A single observation with dimension shape ``state_dim``
        n_possible_actions(``int``): Number of possible actions in current state (obtained using ``env.action_space.n``)
        save_dir(``string``): Path of directory to save RL output.
        h_dict(``dictionary``): A dictionary storing all tunable hyperparameters.
        """

        self.memory = TensorDictReplayBuffer(storage=LazyMemmapStorage(100000, device=torch.device("cpu")))
        self.batch_size = h_dict['batch_size']
        self.learning_mode = h_dict['model']

        self.state_dim = state_dim
        self.actions_count = n_possible_actions
        self.save_dir = save_dir

        self.device = "cuda" if use_cuda else "cpu"

        self.net = DDQL(self.state_dim, self.actions_count).float().to(device=self.device)

        self.exploration_rate = h_dict['exploration_rate']
        self.exploration_rate_decay = h_dict['exploration_rate_decay']
        self.exploration_rate_min = h_dict['exploration_rate_min']
        self.gamma = h_dict['gamma'] #discount factor
        self.optimizer = h_dict['optimiser'](self.net.parameters(), lr=0.00025)
        self.loss_function = torch.nn.SmoothL1Loss()

        self.experience_before_training = 1e4  # min. experiences before training
        self.learn_interval = 3  # no. of experiences between updates to Q_online
        self.sync_interval = 1e4  # no. of experiences between Q_target & Q_online sync

        self.curr_step = 0

        self.save_every = 1e5  # no. of experiences between saving Mario Net


    def act(self, state): #CALLED IN EVERY TRAINING/TESTING EPISODE
        """
        ``state``: the current frame of the game.
        returns int ``action_idx``: the index corresponding to the action which the agent will perform.
        """
        # EPSILON-GREEDY EXPLORATION
        if np.random.rand() < self.exploration_rate:
            action_idx = np.random.randint(self.actions_count)

        # GREEDY EXPLOITATION
        else:
            state = state[0].__array__() if isinstance(state, tuple) else state.__array__()
            state = torch.tensor(state, device=self.device).unsqueeze(0)
            action_values = self.net(state, model="online")
            action_idx = torch.argmax(action_values, axis=1).item()

        # DECAY THE EXPLORATION RATE AFTER EVERY MOVE
        self.exploration_rate *= self.exploration_rate_decay
        
        #AFTER A VARIABLE AMOUNT OF TIME, THE EXPLORATION RATE WILL REACH THE MIN AND WON'T BE DECAYED ANY FURTHER
        self.exploration_rate = max(self.exploration_rate_min, self.exploration_rate)

        # TO REMEMBER HOW MANY TOTAL STEPS MARIO HAS TAKEN
        self.curr_step += 1
        return action_idx

    '''TO RETAIN (STATE, ACTION, STATE', REWARD) MEMORIES'''
    def cache(self, state, next_state, action, reward, done):
        def tuple_check(x):
            return x[0] if isinstance(x, tuple) else x
        
        state = torch.tensor(tuple_check(state).__array__())
        next_state = torch.tensor(tuple_check(next_state).__array__())
        action = torch.tensor([action])
        reward = torch.tensor([reward])
        done = torch.tensor([done])

        self.memory.add(TensorDict({"state": state, "next_state": next_state, "action": action, "reward": reward, "done": done}, batch_size=[]))

    def recall(self):
        """
        RETRIEVE EXPERIENCES FROM MEMORY TO LEARN GAME
        """
        batch = self.memory.sample(self.batch_size).to(self.device)
        
        '''RETRIEVE VALUES FROM MEMORY BUFFER'''
        state, next_state, action, reward, done = (batch.get(key) for key in ("state", "next_state", "action", "reward", "done"))
        
        #transform from tensor size (batch_size, n) into (batch_size,) via squeeze method to ensure that the dimensions are consistent.
        return state, next_state, action.squeeze(), reward.squeeze(), done.squeeze()

    def save(self):
        torch.save(
            dict(model=self.net.state_dict(), exploration_rate=self.exploration_rate),
            save_path
        )
        print(self.learning_mode + f"DDQL network saved. Step: {self.curr_step}")


    def learn(self):
        '''
        CREATE FUNCTIONS ADJUST Q_TARGET AND Q_ONLINE, WHILE CONSISTENTLY SYNCING THE TARGET NETWORK.
        '''
        
        def q_target(self, reward, next_state, done):
            with torch.no_grad(): #because we don't backpropagate on target params.
                next_state_Q = self.net(next_state, model="online")
                best_action = torch.argmax(next_state_Q, axis=1)
                next_Q = self.net(next_state, model="target")[np.arange(0, self.batch_size), best_action]
                return (reward + (1 - done.float()) * self.gamma * next_Q).float()
            
        def q_online_update(self, q_estimate, q_target):
            loss = self.loss_function(q_estimate, q_target)
            #REVERT GRADIENTS TO ZERO BEFORE EVERY BACKPROPAG ITERATION
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            return loss.item()

        if self.curr_step % self.sync_interval == 0:
            '''REPLACE CURRENT TARGET BUFFER WITH ONLINE DICT'''
            self.net.target.load_state_dict(self.net.online.state_dict())

        '''SAVE EVERY SO OFTEN'''
        if self.curr_step % self.save_every == 0:
            self.save()

        if self.curr_step < self.experience_before_training:
            return None, None

        elif self.curr_step % self.learn_interval != 0:
            return None, None
        else:
            '''GRAB EXPERIENCE FROM MEMORY'''
            state, next_state, action, reward, done = self.recall()

            '''ESTIMATE Q VALUE'''
            current_Q_estimate = self.net(state, model="online")[np.arange(0, self.batch_size), action]  # Q_online(s,a)

            '''GRAB Q TARGET'''
            q_target = q_target(reward, next_state, done)

            '''ONLINE NETWORK UPDATES VIA BACKPROPAGATION'''
            loss = q_online_update(current_Q_estimate, q_target)

            return current_Q_estimate.mean().item(), loss

### Logging




#### 4.4 - Hyperparameter Tuning

We tuned model hyperparameters in combination for both DDQL (Table 1) and PPO (Table 2). The environment itself was tuned to different stack sizes (2,4,8) and frame skip rates (2,4,8). Evaluation metrics were either logged and graphed manually due to limited compatibility (in the case of DDQL) or visualised using tensorboard (as in PPO). We focussed on average episode losses and average episode rewards as the two most salient indicators of the model’s success in training the agent.

In [76]:
from tabulate import tabulate
headers = ['Hyperparameter', 'Values Tested']
table_1 = [['Frames Skipped', '4,8,12'], ['Frames Stacked', '2,4,8'],
           ['Gamma (discount factor)', '0.3, 0.5, 0.75, 0.9'], 
           ['Exploration rate', '0.5, 0.1, 1'], ['CNN optimisers', 'ADAM or SGD']]

table_2 = [['Tau (to modulate clipping)', '0.3, 0.5, 0.95'], ['Beta (modulates entropy)', '0.95, 0.99'],
           ['Total global steps (local steps x concurrent environments)', '400, 512 (as used in the literature)'], 
          ]


print('DDQL Manual Parameter Grid Search')
print(tabulate(table_1, headers, tablefmt="grid"))
print()
print('PPO Manual Parameter Grid Search')
print(tabulate(table_2, headers, tablefmt="grid"))

DDQL Manual Parameter Grid Search
+-------------------------+---------------------+
| Hyperparameter          | Values Tested       |
+=========================+=====================+
| Frames Skipped          | 4,8,12              |
+-------------------------+---------------------+
| Frames Stacked          | 2,4,8               |
+-------------------------+---------------------+
| Gamma (discount factor) | 0.3, 0.5, 0.75, 0.9 |
+-------------------------+---------------------+
| Exploration rate        | 0.5, 0.1, 1         |
+-------------------------+---------------------+
| CNN optimisers          | ADAM or SGD         |
+-------------------------+---------------------+

PPO Manual Parameter Grid Search
+------------------------------------------------------------+--------------------------------------+
| Hyperparameter                                             | Values Tested                        |
+============================================================+================

In [77]:
h_dict = {
    'model' : 'DDQL',
    'optimiser' : torch.optim.Adam,
    # 'loss_function' : torch.nn.SmoothL1Loss,
    'exploration_rate' : 0.1,
    'exploration_rate_decay' : 0.99999975,
    'exploration_rate_min' : 0.05,
    'gamma' : 0.5,

    #AFFECTS BOTH:
    'batch_size' : 64,
    #PPO ONLY
    'gamma_ppo' : 0.99,
    'learning_rate' : 0.0003,
    'tau' : 0.95,
    'beta' : 0.01,
    'clipping_epsilon' : 0.2,
    'n_steps' : 2048,
    'n_epochs' : 10,
    'kl_divergence_limit' : None,
    'bool_normalise_advantage' : True,
    'policy' : 'CnnPolicy'
}

#### 4.5 - Instantiate the Environment

In [78]:
RIGHT_RESTRICTED = [["right"], ["right", "A"]]
env = create_environment(version = '3', world = '1', stage = '3', action_space = RIGHT_RESTRICTED, skip = 4, num_stack = 4)
env.reset()
next_state, reward, done, info = env.step(action=0)

#### 4.6 - Train the Models

In [79]:
'''TO USE GPU IF IT EXISTS'''
use_cuda = torch.cuda.is_available()
print(f"Using CUDA: {use_cuda}")

'''PPO TRAINING'''
if h_dict['model'] == 'PPO':
    logs_directory = "./logs/"
    os.makedirs(logs_directory, exist_ok=True) #IF EXISTS, THEN DON'T RAISE ERROR
    
    ppo_model = ppo_learning(env, tensorboard_log = LOG_DIR, h_dict = h_dict)
# 
    ppo_eval_env = create_environment(
        version = '3', world = '1', stage = '3', action_space = RIGHT_RESTRICTED, skip = 4, num_stack = 4
    )
    ppo_eval_callback = EvalCallback(ppo_eval_env, best_model_save_path="./logs/",
                             log_path="./logs/", eval_freq=500,
                             deterministic=True, render=False)
    
    ppo_model.learn(total_timesteps=1000000, callback = ppo_eval_callback)
    ppo_model.save('ppo_model')


else:
    '''DDQL TRAINING'''
    mario = Mario(state_dim=(4, 84, 84), n_possible_actions=env.action_space.n, h_dict = h_dict, use_cuda = use_cuda)
    logger = MetricLogger()

    episodes = 4000
    
    '''INITIALISE METRICS'''
    episode_rewards = []
    episode_avg_q_values = []
    episode_lengths = []
    episode_avg_losses = []

    moving_avg_episode_avg_losses = []
    moving_avg_episode_rewards = []
    moving_avg_episode_lengths = []
    moving_avg_episode_avg_qs = []


    for e in range(episodes):
        '''RESET EPISODE METRICS'''
        start_time = time.time()
        
        curr_episode_q = 0.0
        curr_episode_loss_length = 0
        curr_episode_reward = 0.0
        curr_episode_length = 0
        curr_episode_loss = 0.0
        
        terminal = False
        if e == episodes - 1:
            terminal = True
        state = env.reset()

        while True:
            action = mario.act(state)
            next_state, reward, done, info = env.step(action)
            mario.cache(state, next_state, action, reward, done)
            q_value, loss = mario.learn()
            
            curr_episode_reward += reward
            curr_episode_length += 1
            if loss:
                curr_episode_loss += loss
                curr_episode_q += q
                curr_episode_loss_length += 1

            state = next_state
            
            if done or info["flag_get"]:
                break
        if terminal:
            print('TRAINING COMPLETE!')
            for metric in ["ep_lengths", "ep_avg_losses", "ep_avg_qs", "ep_rewards"]:
                plt.plot(getattr(self, f"moving_avg_{metric}"))
                plt.legend()
                plt.title(f"moving_avg_{metric}")
                plt.show();
                
        '''AT THE END OF EVERY EPISODE ...''' 
        ep_rewards.append(curr_episode_reward)
        ep_lengths.append(curr_episode_length)
        if curr_episode_loss_length == 0: #to avoid value errors
            episode_avg_loss = 0
            episode_avg_q = 0
        else:
            episode_avg_loss = np.round(curr_episode_loss / curr_episode_loss_length, 4)
            episode_avg_q = np.round(curr_episode_q / curr_episode_loss_length, 4)
        episode_avg_losses.append(episode_avg_loss)
        episode_avg_q_values.append(episode_avg_q)
    
        mean_episode_reward = np.round(np.mean(episode_rewards[-100:]), 3)
        mean_episode_length = np.round(np.mean(episode_lengths[-100:]), 3)
        mean_episode_loss = np.round(np.mean(episode_avg_losses[-100:]), 3)
        mean_episode_q = np.round(np.mean(episode_avg_qs[-100:]), 3)
        
        moving_avg_episode_rewards.append(mean_episode_reward)
        moving_avg_episode_lengths.append(mean_episode_length)
        moving_avg_episode_avg_losses.append(mean_episode_loss)
        moving_avg_episode_avg_qs.append(mean_episode_q)

        current_time = self.record_time

        #RECORDS THE LENGTH OF THE EPISODE
        time_since_last_record = np.round(start_time - current_time, 2)

        if episode % 100 == 0 or terminal:
            print(
              f"Episode {episode}\n"
              f"Step {step}\n"
              f"Epsilon {epsilon}\n"
              f"Mean Reward {mean_ep_reward}\n"
              f"Mean Length {mean_ep_length}\n "
              f"Mean Loss {mean_ep_loss}\n "
              f"Mean Q Value {mean_ep_q}\n "
              f"Episode Duration {time_since_last_record}\n "
              f"Time {datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}"
            )

Using CUDA: False


NameError: name 'TensorDictReplayBuffer' is not defined

#### 4.6 - Test & Render

In [85]:
# Load model
if h_dict['model'] == 'PPO':
    model = PPO.load('./ppo_model.zip')
    
else: #i.e. DDQL
    # Load model
    model = None
    # Start the game
    state = env.reset()
    # Loop through the game ``n`` number of times
    n = 500
    #tracks the number of completions
    completion_counter = 0
    #store time to complete each win if Mario won the level
    completion_time = np.array([])

for _ in range(500):
    while True:
        action = model.act(state)
        state, reward, done, info = env.step(action)
        env.render()
    if info['flag_get']:
        completion_counter +=1
        #time is 400 initially...
        completion_time = np.append(completion_time, (400 - info['time']))

    print(f'Completion Rate: {completion_counter / n * 100}%')
    print(f'Average completion time: {np.mean(completion_time)}')


AttributeError: 'NoneType' object has no attribute 'act'

# 5 - Results
Most training was performed on independent devices under different hyperparameter and algorithms for time efficiency. Hence, graphs and rendering are imported into this notebook (rather than directly run on the kernel). 

Despite attempts to tune parameters, the Proximal Policy Optimization (PPO) results were quite erratic. On the other hand, Double Deep Q-Learning (DDQL) outperformed PPO, suggesting that it may be more effective for this task. This is likely due to the complex temporal-state-action space in Super Mario, where DDQL’s dual-network architecture helps in more accurately estimating Q-values, leading to better decision-making. Figure 2 compares the base-case to the best parameters we could find for PPO (learning_rate = 0.001, n_step = 512, epsilon = 0.2, gamma = 0.99, tau = 0.95).

<img src = visuals/ppo_basecase.png width = 600px>(1)<br>
<img src = visuals/ppo_best_result.png width = 600px>(2)<br>
Fig 2. (1) Base case, using parameters from the literature (2) PPO performance with best parameters. 


When experimenting with different stack sizes using the DDQL base case (gamma 0.9 and exploration rate of 1), changing the stack size to 2 or 8 saw a decrease in rewards. It appears that too many previous states may overwhelm the agent and negatively impact its performance, where a stack size of 4 was a point of equilibrium (Figure 3).

<img src = visuals/stacks_compare.png width = 700px> <br>
Fig. 3

We also tested different skip rates, which refer to the number of steps the agent skips before making a decision. We found that a high skip rate led to instability and errors. Skipping too many steps might cause the agent to miss out on important information, leading to suboptimal decision-making and instability in the learning process.

<img src = visuals/steps_compare.png width = 700px> <br>
Fig. 4

The gamma value, which determines the importance of future rewards in the agent’s decision-making process, was also tested. We found that gamma values of 0.5 and 0.75 led to higher rewards compared to the others. However, this also introduced more fluctuations in the agent’s performance, suggesting a trade-off between reward and stability. We committed to 0.5 for it achieved the lowest loss for the number of run episodes.

<img src = visuals/gamma_compare.png width = 700px> <br>
Fig. 5


Finally, we tested different exploration rates 1, 0.1, and 0.05 to balance between exploration and exploitation. A higher exploration rate led to more diverse experiences but also more mistakes, resulting in a higher average loss. However, an exploration rate of 0.05, despite a higher initial average loss, showed a decreasing trend over the training term.

<img src = visuals/ex_rate_compare.png width = 700px> <br>
Fig. 6

In the end, we achieved a successful run with the following parameters: Stack 4, Skip 4, Gamma 0.5, and Exploration Rate: 0.1 (Figure 7).

<img src = visuals/best_result.png width = 600px><br>
Fig 7. (1) Average episode rewards maintain consistently high and (2) episode losses are steadily decreasing with each iteration. 


# 6 - Discussion

Overall, our project demonstrates the potential of DDQL in training a game-playing agent and could be replicable in other domains. The key strengths of our solution are rooted in our pre-processing and selection of optimal hyperparameters utilising literature and novel combinations. Firstly, our downscaling of a [3, 240, 256] environment frame to only [1, 84, 84] significantly boosts training time through the computational bottleneck that is a CNN (in DDQL) or PPO algorithm. Additionally, by using multiple CPUs/GPUs to run concurrent training processes and saving models for re-training, we have further boosted its effectiveness. Lastly, the fact that DDQL is relatively new, our proposed model can inspire future studies to improve efficiency. 

The key weakness of our solution is its isolation to World 1.1, as reinforcement learning models themselves are domain-specific and thus must be retrained to different scenarios, unlike other ANNs, which can be pre-trained using different datasets. Additionally, the algorithm is meticulous in implementing and tuning parameters. Ultimately, future work includes:

- Training on all 3 worlds of the game.
- Using evolutionary methods combined with our solution to take the best of two worlds [12]
- Optimising the parameters of DDQL for different worlds or levels within the Super Mario Bros gym environment.
- Simplifying the implementation of DDQL (using a library instead, such as stable-baselines-2)
- Varying the reward function for different objectives (e.g. a killer Mario whose goal is to kill enemies rather than obtain coins or beat the level quickly).

# REFERENCES

[1] R. Sutton and A. Barto, Reinforcement learning: An introduction, vol. 1. Cambridge Univ Press, 1998.<br>
[2] Watkins and Dayan, Q-learning.Machine Learning. 1992.<br>
[3] Singhal, Kartik et al. “Deep Reinforcement Learning in Mario.” 2018.<br>
[4] Mauro Comi, How to teach AI to play Games: Deep Reinforcement Learning, https://towardsdatascience.com/how-to-teach-an-ai-to-play-games-deep-reinforcement-learning-28f9b920440a, 2018.<br>
[5] Artem Oppermann, An Introduction to Double Deep Q-Learning, https://builtin.com/artificial-intelligence/double-deep-q-learning, 2022.<br>
[6] Dilith Jayakody, Double Deep Q-Networks (DDQN) – A Quick Intro (with Code), https://dilithjay.com/blog/ddqn/, 2022.<br>
[7] Reinisch, N.; Rudolph, F.; Günther, S.; Bailly, D.; Hirt, G. Successful Pass Schedule Design in Open-Die Forging Using Double Deep Q-Learning. Processes 2021, 9, 1084.<br>
[8] Andrew Grebenisan, Play Super Mario Bros with a Double Deep Q-Network, https://blog.paperspace.com/building-double-deep-q-network-super-mario-bros/, 2020.<br>
[9] Brandon Da Silva, Playing Super Mario Bros with Proximal Policy Optimization, https://brandinho.github.io/mario-ppo/, 2018.<br>
[10] Amjad Yousef Majid, Serge Saaybi, Tomas van Rietbergen, Vincent Francois-Lavet, R Venkatesha Prasad, Chris Verhoeven, Deep Reinforcement Learning Versus Evolution Strategies: A Comparative Survey, https://arxiv.org/pdf/2110.01411.pdf, 2021.<br>
[11] Hado van Hasselt and Arthur Guez and David Silver, Deep Reinforcement Learning with Double Q-learning, https://arxiv.org/pdf/1509.06461.pdf, 2015.<br>
[12] John Schulman and Filip Wolski and Prafulla Dhariwal and Alec Radford and Oleg Klimov, Proximal Policy Optimization Algorithms, https://arxiv.org/abs/1707.06347, 2017.<br>
[13] F. Moreno-Vera, "Performing Deep Recurrent Double Q-Learning for Atari Games," 2019 IEEE Latin American Conference on Computational Intelligence (LA-CCI), Guayaquil, Ecuador, 2019, pp. 1-4, doi: 10.1109/LA-CCI47412.2019.9036763.<br>
[14] Sean Klein, CS229 Final Report Deep Q-Learning to Play Mario, https://cs229.stanford.edu/proj2016/report/klein-autonomousmariowithdeepreinforcementlearning-report.pdf , 2016.<br>
[15] Christian Kauten, gym-super-mario-bros 7.4.0, https://pypi.org/project/gym-super-mario-bros/, 2022.<br>